## 第六章 支持向量机
* https://www.toutiao.com/a6698315325579985420/  
* https://blog.csdn.net/github_38325884/article/details/74418365  
* 可以做分类也可以做回归：分类（sklearn.svm.SVC），回归（sklearn.svm.SVR）。  
* 支持向量机主要用在分类上，回归问题本章不予讨论，感兴趣的话可以参阅下边文献:
https://blog.csdn.net/lpsl1882/article/details/52411987  
https://blog.csdn.net/qq_34531825/article/details/52891780

### 一、支持向量机方法概述  
#### 1.支持向量机介绍  
* 支持向量机方法本质上是二分类问题，但sklearn中的SVM函数实现了多分类算法，也就是多次利用二分类方法实现多分类。
* 支持向量机分为：线性可分支持向量机算法（数据必须可分，否则方法不能用）、线性支持向量机算法（数据是否可分都可以用）、非线性支持向量机算法。
* 支持向量机算法就是利用核函数实现对原数据进行高维映射，使得原数据在低维度空间不能使用线性支持向量机，但映射到高维空间的数据基本线性可分，就可以使用线性支持向量机进行分类。
* 核函数一般和应用场景有关，有专门针对特定应用领域进行核函数开发和建模的科研人员在从事这方面的研究。但实际上也有一些通用的'万金油'核函数：多项式核函数和高斯核函数。选择核函数的一般原则：若特征个数比样本点数还多，则选取线性核函数。如果数据量很大，选择高斯核或多项式核。
* 支持向量算法调参需要好好的熟悉一下，了解需要调整的参数以及每个参数对分类效果的影响。

#### 2. 关于支持向量机svm.SVC()中的参数说明：  
* (1) 高斯核函数  
svm.SVC(C=1.0, kernel='rbf', gamma=0.01),可调参数有惩罚项C和gamma，gamma就是模型中的1/(2*sigma^2)  
参数gamma控制着模型的复杂程度，这个值越大，模型越复杂，但容易出现过拟合；值越小，模型就越精简，但容易出现欠拟合。  
参数C是惩罚项，C值越大意味着要求分类错误尽可能小，C越小意味着可以有更大的错误容忍。   
* (2) 多项式核函数  
svm.SVC(C=1.0, kernel='poly', degree=2)   #多项式核函数，可调参数有惩罚项C和多项式次数degree。  
多项式次数degree越大，模型越复杂，非线性分类能力越强，但容易出现过拟合。  
* (3) 线性核函数  
svm.SVC(C=1.0, kernel='linear')        #线性核函数，可调参数只有惩罚项C。  

### 二、实例

#### 实例：基于支持向量机方法的泰坦尼克号幸存者预测
* 数据来源：https://www.kaggle.com/c/titanic/data  
　　1912年4月15日，在首次航行期间，泰坦尼克号撞上冰山后沉没，2224名乘客和机组人员中有1502人遇难。这场轰动的悲剧震惊国际社会，在这次海难中导致死亡率高的原因之一是没有足够的救生艇给乘客和机组人员，虽然幸存下来有一部分的运气因素，但是还是有一些人比其他人的生存下来的可能性更高，比如妇女、儿童和上层阶级的人士。在这个学习之中，我们将用支持向量机方法来预测一些人生存的可能性。用机器学习来预测哪些乘客能更幸免于难。  
　　数据集有891个训练样本，每个样本有11个特征，一个标签列Survived。数据集有缺失值，有字符型变量，因此数据需要做预处理。
* 数据特征说明：  
PassengerId: 乘客的ID，对预测没有用处  
Survived：1代表幸存，0代表遇难.  
Pclass：舱位等级，是很重要的特征。看过电影的读者知道，高仓位等级的乘客能更快的到达甲板，从而更容易获救。  
Name：乘客名字，这个特征与幸存无关，对也侧没有用处。  
Sex：乘客性别，看过电影的知道，因为救生艇数量不够，船长让妇女和儿童先上救生艇。这个是重要的特征。  
Age：乘客年龄，儿童会优先上救生艇。  
SibSp：兄弟姐妹及配偶同在船上的个数  
Parch：父母或子女的个数  
Ticket：船票号，不使用这个特征。  
Fare：船票价格  
Cabin：乘客所在船舱号。最早被水淹没的船舱位置,乘客的幸存概率要低一些。但这个特征的数据大量丢失,舍弃这个特征。  
Embarked：乘客登船的港口。  

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

data=pd.read_csv('./data_picture/chapter6/titanic.csv',index_col=0)
data.drop(['Name','Ticket','Cabin'],axis=1,inplace=True)
data.dropna(how='any',inplace=True)   #删除有缺失值所在的整行
data=pd.get_dummies(data, columns=['Sex','Embarked'])
data.head()

,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
PassengerId,,,,,,,,,,,
1,0,3,22.0,1,0,7.2500,0,1,0,0,1
2,1,1,38.0,1,0,71.2833,1,0,1,0,0
3,1,3,26.0,0,0,7.9250,1,0,0,0,1
4,1,1,35.0,1,0,53.1000,1,0,0,0,1
5,0,3,35.0,0,0,8.0500,0,1,0,0,1


In [2]:
#2.生成训练集和测试集
X=data.drop('Survived',axis=1)
y=data['Survived']

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=33)
print(y_train.value_counts(ascending=True,dropna=True))   #打印训练集各类别训练样本个数

1    220
0    314
Name: Survived, dtype: int64


In [3]:
#3.模型训练（此处可以利用交叉验证的方法选取超参数）
from sklearn import svm
from sklearn.model_selection import GridSearchCV  ##遍历验证函数
from sklearn.model_selection import ShuffleSplit  #生成交叉验证集的函数调入

#参数gamma控制着模型的复杂程度，这个值越大，模型越复杂，但容易出现过拟合；值越小，模型就越精简，但容易出现欠拟合。
#参数C是惩罚项，C值越大意味着要求分类错误尽可能小，C越小意味着可以有更大的错误容忍。
model_svm=svm.SVC(kernel='rbf')
par_grid = {"C":[0.5,1,1.5,2,2.5,3,4], "gamma": [0.01,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]}  #遍历参数网格
cv = ShuffleSplit(n_splits=5, test_size=0.2)
grid_search = GridSearchCV(model_svm, param_grid=par_grid, cv=cv)
grid_search.fit(X_train, y_train)
print('Best score:',grid_search.best_score_)   #这个得分和未来对测试集的预测精度差不多。
print('Best parameters:',grid_search.best_params_)

model=grid_search.best_estimator_  #将最佳分类器赋予model
'''  也可以用这种方式重新构造模型，但需要重新训练模型。
c_value=grid_search.best_params_['C']  #取出最佳C
gamma_value=grid_search.best_params_['gamma']  #取出最佳gamma
model=svm.SVC(C=c_value,kernel='rbf',gamma=gamma_value)
model.fit(X_train,y_train)  #y_train可以是行向量也可以是列向量，knn会自动将y_train转换为列向量。
'''
#4.分类模型效果评估
from sklearn import metrics
from sklearn.metrics import classification_report
model_train_accuracy=model.score(X_train, y_train)      #训练集的准确率
model_test_accuracy=model.score(X_test, y_test)         #测试集的准确率
y_predict=model.predict(X_test)                        #预测新的数据分类,返回类别一维数组
print('--------------------------------------------------------------')
print('训练集准确率:',model_train_accuracy)       
print('测试集准确率:',model_test_accuracy)  
print('--------------------------------------------------------------')
model_report=classification_report(y_test,y_predict)
print(model_report)

Best score: 0.7401869158878505
Best parameters: {'C': 4, 'gamma': 0.01}
--------------------------------------------------------------
训练集准确率: 0.850187265917603
测试集准确率: 0.7415730337078652
--------------------------------------------------------------
              precision    recall  f1-score   support

           0       0.80      0.77      0.79       110
           1       0.65      0.69      0.67        68

    accuracy                           0.74       178
   macro avg       0.73      0.73      0.73       178
weighted avg       0.74      0.74      0.74       178

